In [ ]:
# models/neural/mlp.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from typing import Dict, Any
from sklearn.preprocessing import StandardScaler, LabelEncoder

from ..base_classifier import BaseClassifier

class MLPClassifier(BaseClassifier):
    """Multi-Layer Perceptron classifier using PyTorch"""
    
    def __init__(self, config: Dict[str, Any]):
        super().__init__(config)
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def _create_model(self, input_dim: int, n_classes: int) -> nn.Module:
        """Create MLP model"""
        hidden_dims = self.config.get('hidden_dims', [512, 256, 128])
        dropout_rate = self.config.get('dropout_rate', 0.3)
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                nn.BatchNorm1d(hidden_dim)
            ])
            prev_dim = hidden_dim
            
        layers.append(nn.Linear(prev_dim, n_classes))
        
        return nn.Sequential(*layers)
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> 'MLPClassifier':
        """Train MLP classifier"""
        # Preprocess data
        X_scaled = self.scaler.fit_transform(X)
        y_encoded = self.label_encoder.fit_transform(y)
        
        # Convert to tensors
        X_tensor = torch.FloatTensor(X_scaled).to(self.device)
        y_tensor = torch.LongTensor(y_encoded).to(self.device)
        
        # Create model
        input_dim = X_scaled.shape[1]
        n_classes = len(np.unique(y_encoded))
        self.model = self._create_model(input_dim, n_classes).to(self.device)
        
        # Training setup
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(
            self.model.parameters(),
            lr=self.config.get('learning_rate', 0.001),
            weight_decay=self.config.get('weight_decay', 1e-4)
        )
        
        # Create data loader
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(
            dataset,
            batch_size=self.config.get('batch_size', 32),
            shuffle=True
        )
        
        # Training loop
        n_epochs = self.config.get('n_epochs', 100)
        self.model.train()
        
        for epoch in range(n_epochs):
            total_loss = 0
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            
            if epoch % 20 == 0:
                avg_loss = total_loss / len(dataloader)
                print(f"Epoch {epoch}/{n_epochs}, Loss: {avg_loss:.4f}")
        
        self.is_fitted = True
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """Make predictions"""
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before making predictions")
            
        X_scaled = self.scaler.transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(self.device)
        
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()
            
        return self.label_encoder.inverse_transform(predictions)
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """Return prediction probabilities"""
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before making predictions")
            
        X_scaled = self.scaler.transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(self.device)
        
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
            
        return probabilities
    
    def get_feature_importance(self) -> np.ndarray:
        """Return feature importance (gradient-based)"""
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before getting feature importance")
            
        # Use gradient-based feature importance
        dummy_input = torch.randn(1, self.scaler.n_features_in_).to(self.device)
        dummy_input.requires_grad_(True)
        
        self.model.eval()
        output = self.model(dummy_input)
        output.backward(torch.ones_like(output))
        
        importance = torch.abs(dummy_input.grad).mean(dim=0).cpu().numpy()
        return importance